In [2]:
import pyawr.mwoffice as mwo

import numpy as np
from numpy.polynomial.polynomial import polyfit

import matplotlib.pyplot as plt

import qkit.analysis.circle_fit.circle_fit_2019.circuit as cf

#***PICK ONE***
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
# %matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
# %matplotlib tk 

ModuleNotFoundError: No module named 'qkit'

In [ ]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #In/voke circuit simulator analysis

In [ ]:
def reset_freqs (l_bnd=4e9, u_bnd=8e9, steps=10000):
    '''
    Reset the MWO project frequencies.

    returns an array running from 
    lower_bound to upper_bound in steps steps.
    '''
    
    awrde.Project.Frequencies.Clear() # clear the frequencies specified for the project
    freq_arr = np.linspace(l_bnd, u_bnd, steps)
    awrde.Project.Frequencies.AddMultiple(freq_arr) # add the frequencies in the passed frequency array

    awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return freq_arr

In [ ]:
def set_circ_params(circ_name='Sample_Subcircuit', get_params=False, **kwargs): 
    '''Sets specified circuit parameters in circuit called circ_name.
    

    If you attempt to pass a parameter that doesn't exit in the circuit called circ_name, 
    MWO will throw an error.

    Returns a dictionary with the new values.
    '''

    passed_circ_param_vals = { # dictionary of subcircuit element parameter values (e.g. the value of the capacitor (element's) capacitance (parameter.))
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2).ValueAsDouble,
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2).ValueAsDouble,
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2).ValueAsDouble
    }
    passed_circ_params = { # dictionary of subcircuit component 
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2),
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2),
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2)
    }

    # print("DEBUG: Setting:" + kwargs.__str__())

    new_circ_param_vals = {**passed_circ_param_vals, **kwargs} # in the case of duplicate keys, only the later key-value pair is preserved
    # print("DEBUG: new parameters:" + new_circ_param_vals.__str__())

    for i, value in enumerate(new_circ_param_vals.values()):
        list(passed_circ_params.values())[i].ValueAsDouble = value # in python3, dict.keys(), .values(), and .items() return dynamically changing view objects, but not the objects themselves. Hence, list().

        # print("DEBUG: " + list(passed_circ_param_vals)[i] + " set to " + str(value))
    
    if not(get_params):
        awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return new_circ_param_vals

In [ ]:
def create_polar_graph(schem_name='Hanger_Test'):
    graphs = awrde.Project.Graphs
    # TODO: Name helper graph appropriately
    graph = graphs.Add("CalcHelper {}".format(schem_name), mwo.mwGraphType.mwGT_Polar)

    graph.Measurements.Add(schem_name, 'S(2,1)')

    return graph

In [ ]:
def delete_graph(graph=None):
    for i in range(awrde.Project.Graphs.Count):
        if awrde.Project.Graphs.Item(i+1).Name == graph.Name:
            ret = awrde.Project.Graphs.Remove(i+1)

    return ret

In [ ]:
def get_meas_vals(graph=None):
    #TODO: iterate over all measurement indicies, add a try-catch for when there's only 1 measurement on the graph
    meas = graph.Measurements[0]

    trace = meas.TraceValues(1)
    freqs, real, imag = zip(*trace)

    return (freqs, real, imag)

In [ ]:
def plot_polar(re, im):
    print(re)
    print(im)

    plt.scatter(re, im)

In [ ]:
def plot_and_fit():

    gr = create_polar_graph()
    awrde.Project.Simulator.Analyze()
    meas_vals = get_meas_vals(gr)
    freqs, real, imag = np.asarray(meas_vals)
    plot_polar(real, imag)
    cpx = real + imag*1j
    delete_graph(gr)

    circ = cf.reflection_port(freqs, cpx)
    circ.autofit()
    circ.plotall()
    print(circ.fitresults)

    return circ

In [ ]:
def is_fit_good(circ):
    fit_good = False

    if circ.Ql>0 and circ.Qc>0 and circ.Qi>0:
        print("Qs all positive.")

        if circ.f_data[0] < circ.fr < circ.f_data[-1]:
            print("fr in f_data")
            fit_good = True
        
        else:
            fit_good = False

    else:
        fit_good = False
    
    return fit_good

In [ ]:
from sympy import Symbol, diff, exp, cos, pi, lambdify

class analyze_circuit(cf.circuit):
    '''
    Circlefit class for finding the exact derivative of Sij and maximizing it.
    '''

    n_ports = None
    f_data = None

    def __init__(self, circ):
        self.n_ports = circ.n_ports
        self.f_data = circ.f_data

    def sensitive_Sij(self, fr, Ql, Qc, phi=0., a=1., alpha=0., delay=0.):
        '''
        Returns the frequency at which the magnitude of the derivative of Sij is maximized.
        '''

        f = Symbol('f', real=True)

        complexQc = Qc*cos(phi)*exp(-1j*phi)
        S = a*exp(1j*(alpha-2*pi*f*delay)) * (
            1. - 2.*Ql / (complexQc * self.n_ports * (1. + 2j*Ql*(f/fr-1.)))
        )

        ds = diff(S, f)
        ds_np = lambdify(f, ds, 'numpy')
        #NOTE: I'm just maximizing the magnititude of the derivatives of Sij here... Complex numbers have no ordering
        ds_mags = np.asarray(np.abs(ds_np(self.f_data)))
        arg = np.argmax(ds_mags)

        return self.f_data[arg]




In [ ]:
def fit_circ(circ_type='reflection_port'):

    gr = create_polar_graph()
    awrde.Project.Simulator.Analyze()
    meas_vals = get_meas_vals(gr)
    freqs, real, imag = np.asarray(meas_vals)
    plot_polar(real, imag)
    cpx = real + imag*1j
    delete_graph(gr)

    if circ_type == 'reflection_port':
        circ = cf.reflection_port(freqs, cpx)
        circ.autofit()
        circ.plotall()
        print(circ.fitresults)
    
    elif circ_type == 'notch_port':
        circ = cf.notch_port(freqs, cpx)
        circ.autofit()
        circ.plotall()
        print(circ.fitresults)

    else:
        print("Invalid circuit measurement mode! Must be 'reflection' or 'notch'.")

    return circ

In [ ]:
reset_freqs()
circ = plot_and_fit()
is_fit_good(circ)

NameError: name 'reset_freqs' is not defined